In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [2]:
all_transactions = feather.read_dataframe('all_transactions')

In [3]:
all_transactions.drop('cat2_dum_nan',axis=1,inplace=True) 

In [5]:
all_transactions['category_combined_without_cat1'] = (all_transactions['cat2_dum_0.0'] * 1) + (all_transactions['cat2_dum_1.0'] * 2) + (all_transactions['cat2_dum_2.0'] * 3) + (all_transactions['cat2_dum_3.0'] * 4) + (all_transactions['cat2_dum_4.0'] * 5) + (all_transactions['cat2_dum_5.0'] * 6)  + (all_transactions['cat3_dum_A'] * 0)  + (all_transactions['cat3_dum_B'] * 6)  + (all_transactions['cat3_dum_C'] * 12)  + (all_transactions['cat3_dum_nan'] * 18)

In [5]:
all_transactions['category_combined_with_cat1'] = (all_transactions['cat2_dum_0.0'] * 1) + (all_transactions['cat2_dum_1.0'] * 2) + (all_transactions['cat2_dum_2.0'] * 3) + (all_transactions['cat2_dum_3.0'] * 4) + (all_transactions['cat2_dum_4.0'] * 5) + (all_transactions['cat2_dum_5.0'] * 6)  + (all_transactions['cat3_dum_A'] * 0)  + (all_transactions['cat3_dum_B'] * 6)  + (all_transactions['cat3_dum_C'] * 12)  + (all_transactions['cat3_dum_nan'] * 18) + (all_transactions['category_1'] * 24)

In [7]:
####groupped super category ####

In [19]:
def modeplus(x): return mode(x)[0]
def top_quarter_quantile(x): return x.quantile(q=0.25,interpolation='lower')
def bottom_quarter_quantile(x): return x.quantile(q=0.75,interpolation='higher')

In [21]:
groupped = all_transactions.groupby(['card_id']).agg({'purchase_date_day':['max','min','nunique'],
                                                      'purchase_amount':'sum',
                                                      'category_combined_without_cat1':[modeplus,top_quarter_quantile,bottom_quarter_quantile,'std'],
                                                      'category_combined_with_cat1':[modeplus,top_quarter_quantile,bottom_quarter_quantile,'std'],})

In [27]:
groupped.columns = ['_'.join(x) for x in groupped]

In [28]:
groupped.reset_index(inplace=True)

In [30]:
groupped['purchase_date_day_ptp'] = groupped.purchase_date_day_max - groupped.purchase_date_day_min

In [32]:
del groupped['purchase_date_day_max']
del groupped['purchase_date_day_min']

In [34]:
groupped['average_purchase_per_day'] = groupped.purchase_amount_sum / groupped.purchase_date_day_ptp
groupped['average_purchase_per_unique_day'] = groupped.purchase_amount_sum / groupped.purchase_date_day_nunique

In [35]:
del groupped['purchase_amount_sum']
del groupped['purchase_date_day_ptp']
del groupped['purchase_date_day_nunique']

In [37]:
feather.write_dataframe(groupped,'cats and percent spent\\groupped_table')

In [6]:
#### average per category and city ####

In [7]:
custom_rel_to_cats = pd.DataFrame()

In [8]:
for x in range(7,all_transactions.category_combined_without_cat1.max()):
    custom_rel_to_cats[f'installments_catwitho_{x}'] = (all_transactions.category_combined_without_cat1 == x).astype(np.float32).replace({0:np.nan}) * all_transactions.installments.values
    custom_rel_to_cats[f'installments_catwitho_{x}'] = custom_rel_to_cats[f'installments_catwitho_{x}'].values / custom_rel_to_cats[f'installments_catwitho_{x}'].mean()

C:\Users\User\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
custom_purch_to_cats = pd.get_dummies(all_transactions.category_combined_without_cat1,prefix='purch_cats_combo')

In [13]:
for x in custom_purch_to_cats:
    custom_purch_to_cats[x] = (custom_purch_to_cats[x].values * all_transactions.purchase_amount.values)
    custom_purch_to_cats[x].replace({0:np.nan},inplace=True)
    custom_purch_to_cats[x] = custom_purch_to_cats[x].values / custom_purch_to_cats[x].median()

In [17]:
custom_purch_to_city = pd.get_dummies(all_transactions.city_id,prefix='purch_cats_cityid')

In [18]:
for x in custom_purch_to_city:
    custom_purch_to_city[x] = (custom_purch_to_city[x].values * all_transactions.purchase_amount.values)
    custom_purch_to_city[x].replace({0:np.nan},inplace=True)
    custom_purch_to_city[x] = custom_purch_to_city[x].values / custom_purch_to_city[x].median()

In [22]:
card_id = all_transactions.card_id

In [23]:
del all_transactions

In [31]:
for df in [custom_purch_to_city,custom_rel_to_cats,custom_purch_to_cats]:
    df['card_id'] = card_id

In [2]:
custom_purch_to_city = feather.read_dataframe('purch_rel\\custom_purch_to_city')

In [5]:
custom_purch_to_city = custom_purch_to_city.groupby(['card_id']).agg({x:['min','mean','max','sum','std','skew'] for x in custom_purch_to_city if x != 'card_id'})

In [6]:
custom_rel_to_cats = feather.read_dataframe('purch_rel\\custom_rel_to_cats')

In [7]:
custom_rel_to_cats = custom_rel_to_cats.groupby(['card_id']).agg({x:['min','mean','max','sum','std','skew'] for x in custom_rel_to_cats if x != 'card_id'})

In [8]:
custom_purch_to_cats = feather.read_dataframe('purch_rel\\custom_purch_to_cats')

In [9]:
custom_purch_to_cats = custom_purch_to_cats.groupby(['card_id']).agg({x:['min','mean','max','sum','std','skew'] for x in custom_purch_to_cats if x != 'card_id'})

In [12]:
for df in [custom_purch_to_cats,custom_rel_to_cats,custom_purch_to_city]:
    df.columns = ['_'.join(x) for x in df]
    df.reset_index(inplace=True)

In [25]:
all_groupped = pd.merge(custom_purch_to_cats,custom_rel_to_cats,on='card_id',how='left')

In [26]:
all_groupped = pd.merge(all_groupped,custom_purch_to_city,on='card_id',how='left')

In [27]:
feather.write_dataframe(all_groupped,'purch_rel\\all_groupped')

In [ ]:
############ shopping dates ############

In [4]:
all_transactions['holiday_New_Year_2017']=(pd.to_datetime('2017-01-01')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -4 and x < 1 else 0)

all_transactions['holiday_Carnival_2017']=(pd.to_datetime('2017-02-28')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -4 and x < 1 else 0)

all_transactions['holiday_Good_Friday_2017']=(pd.to_datetime('2017-04-14')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Easter_2017']=(pd.to_datetime('2017-04-16')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Tirandes_Day_2017']=(pd.to_datetime('2017-04-21')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Labor_Day_2017']=(pd.to_datetime('2017-05-01')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Mothers_Day_2017']=(pd.to_datetime('2017-05-14')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Valentine_Day_2017']=(pd.to_datetime('2017-06-12')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Corpus_Christi_2017']=(pd.to_datetime('2017-06-15')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -2 and x < 0 else 0)

all_transactions['holiday_Rebelion_day_2017']=(pd.to_datetime('2017-07-09')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -2 and x < 0 else 0)

all_transactions['holiday_Fathers_day_2017']=(pd.to_datetime('2017-08-13')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -2 and x < 0 else 0)

all_transactions['holiday_Independence_day_2017']=(pd.to_datetime('2017-09-07')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Children_day_2017']=(pd.to_datetime('2017-10-12')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_All_Souls_day_2017']=(pd.to_datetime('2017-11-02') - all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Proclamation_of_Rep_2017']=(pd.to_datetime('2017-11-15') - all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Black_Friday_2017']=(pd.to_datetime('2017-11-24') - all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -5 and x < 2 else 0)

all_transactions['holiday_Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)


all_transactions['holiday_New_Year_2018']=(pd.to_datetime('2018-01-01')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -4 and x < 1 else 0)

all_transactions['holiday_Carnival_2018']=(pd.to_datetime('2018-02-13')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -4 and x < 1 else 0)

all_transactions['holiday_Good_Friday_2018']=(pd.to_datetime('2018-04-30')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Easter_2018']=(pd.to_datetime('2018-04-01')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Tirandes_Day_2018']=(pd.to_datetime('2018-04-21')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Labor_Day_2018']=(pd.to_datetime('2018-05-01')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

all_transactions['holiday_Mothers_Day_2018']=(pd.to_datetime('2018-05-13')-all_transactions['purchase_date']).dt.days.apply(lambda x: x if x > -3 and x < 0 else 0)

In [5]:
all_transactions['all_holidays'] = 0
for x in [x for x in all_transactions if re.match('holiday_',x)]:
    all_transactions.all_holidays += all_transactions[x]
    all_transactions[x] = all_transactions[x].astype(np.bool_).values * all_transactions.purchase_amount.values

all_transactions['all_holidays'] = all_transactions['all_holidays'].astype(np.bool_) * all_transactions.purchase_amount.values

In [7]:
to_drop = [x for x in all_transactions if re.match('holiday_',x)]
to_drop += ['card_id','purchase_date_day','purchase_amount','all_holidays']
to_drop = list(set(all_transactions.columns) - set(to_drop))

In [11]:
for x in to_drop:
    del all_transactions[x]

In [32]:
def bool_purchase_percent(x):
    return (x > 0).sum() / x.size

def ptp_plus(x):
    return x.ptp()

In [50]:
for x in [x for x in all_transactions if re.match('holiday_',x)]:
    all_transactions[x] = all_transactions[x].replace({0:np.nan})

In [55]:
all_transactions['day_with_holiday_purch'] = all_transactions.all_holidays.astype(np.bool_) * all_transactions.purchase_date_day

In [57]:
all_transactions.day_with_holiday_purch = all_transactions.day_with_holiday_purch.replace({0:np.nan})
all_transactions.all_holidays = all_transactions.all_holidays.replace({0:np.nan})

In [60]:
agg_parms = {x:['min','sum','max','mean','std',bool_purchase_percent] for x in all_transactions if re.match('holiday_',x)}
agg_parms['day_with_holiday_purch'] = 'nunique'
agg_parms['purchase_date_day'] = ptp_plus
agg_parms['purchase_amount'] = ['min','sum','max','mean','size']
agg_parms['all_holidays'] = ['min','sum','max','mean','std']

In [61]:
groupped = all_transactions.groupby(['card_id']).agg(agg_parms)

In [62]:
groupped.columns = ['_'.join(x) for x in groupped]

In [63]:
groupped.reset_index(inplace=True)

In [64]:
groupped.head()

,card_id,holiday_New_Year_2017_min,holiday_New_Year_2017_sum,holiday_New_Year_2017_max,holiday_New_Year_2017_mean,holiday_New_Year_2017_std,holiday_New_Year_2017_bool_purchase_percent,holiday_Carnival_2017_min,holiday_Carnival_2017_sum,holiday_Carnival_2017_max,holiday_Carnival_2017_mean,holiday_Carnival_2017_std,holiday_Carnival_2017_bool_purchase_percent,holiday_Good_Friday_2017_min,holiday_Good_Friday_2017_sum,holiday_Good_Friday_2017_max,holiday_Good_Friday_2017_mean,holiday_Good_Friday_2017_std,holiday_Good_Friday_2017_bool_purchase_percent,holiday_Easter_2017_min,holiday_Easter_2017_sum,holiday_Easter_2017_max,holiday_Easter_2017_mean,holiday_Easter_2017_std,holiday_Easter_2017_bool_purchase_percent,holiday_Tirandes_Day_2017_min,holiday_Tirandes_Day_2017_sum,holiday_Tirandes_Day_2017_max,holiday_Tirandes_Day_2017_mean,holiday_Tirandes_Day_2017_std,holiday_Tirandes_Day_2017_bool_purchase_percent,holiday_Labor_Day_2017_min,holiday_Labor_Day_2017_sum,holiday_Labor_Day_2017_max,holiday_Labor_Day_2017_mean,holiday_Labor_Day_2017_std,holiday_Labor_Day_2017_bool_purchase_percent,holiday_Mothers_Day_2017_min,holiday_Mothers_Day_2017_sum,holiday_Mothers_Day_2017_max,holiday_Mothers_Day_2017_mean,holiday_Mothers_Day_2017_std,holiday_Mothers_Day_2017_bool_purchase_percent,holiday_Valentine_Day_2017_min,holiday_Valentine_Day_2017_sum,holiday_Valentine_Day_2017_max,holiday_Valentine_Day_2017_mean,holiday_Valentine_Day_2017_std,holiday_Valentine_Day_2017_bool_purchase_percent,holiday_Corpus_Christi_2017_min,holiday_Corpus_Christi_2017_sum,holiday_Corpus_Christi_2017_max,holiday_Corpus_Christi_2017_mean,holiday_Corpus_Christi_2017_std,holiday_Corpus_Christi_2017_bool_purchase_percent,holiday_Rebelion_day_2017_min,holiday_Rebelion_day_2017_sum,holiday_Rebelion_day_2017_max,holiday_Rebelion_day_2017_mean,holiday_Rebelion_day_2017_std,holiday_Rebelion_day_2017_bool_purchase_percent,holiday_Fathers_day_2017_min,holiday_Fathers_day_2017_sum,holiday_Fathers_day_2017_max,holiday_Fathers_day_2017_mean,holiday_Fathers_day_2017_std,holiday_Fathers_day_2017_bool_purchase_percent,holiday_Independence_day_2017_min,holiday_Independence_day_2017_sum,holiday_Independence_day_2017_max,holiday_Independence_day_2017_mean,holiday_Independence_day_2017_std,holiday_Independence_day_2017_bool_purchase_percent,holiday_Children_day_2017_min,holiday_Children_day_2017_sum,holiday_Children_day_2017_max,holiday_Children_day_2017_mean,holiday_Children_day_2017_std,holiday_Children_day_2017_bool_purchase_percent,holiday_All_Souls_day_2017_min,holiday_All_Souls_day_2017_sum,holiday_All_Souls_day_2017_max,holiday_All_Souls_day_2017_mean,holiday_All_Souls_day_2017_std,holiday_All_Souls_day_2017_bool_purchase_percent,holiday_Proclamation_of_Rep_2017_min,holiday_Proclamation_of_Rep_2017_sum,holiday_Proclamation_of_Rep_2017_max,holiday_Proclamation_of_Rep_2017_mean,holiday_Proclamation_of_Rep_2017_std,holiday_Proclamation_of_Rep_2017_bool_purchase_percent,holiday_Black_Friday_2017_min,holiday_Black_Friday_2017_sum,holiday_Black_Friday_2017_max,holiday_Black_Friday_2017_mean,holiday_Black_Friday_2017_std,holiday_Black_Friday_2017_bool_purchase_percent,holiday_Christmas_Day_2017_min,holiday_Christmas_Day_2017_sum,holiday_Christmas_Day_2017_max,holiday_Christmas_Day_2017_mean,holiday_Christmas_Day_2017_std,holiday_Christmas_Day_2017_bool_purchase_percent,holiday_New_Year_2018_min,holiday_New_Year_2018_sum,holiday_New_Year_2018_max,holiday_New_Year_2018_mean,holiday_New_Year_2018_std,holiday_New_Year_2018_bool_purchase_percent,holiday_Carnival_2018_min,holiday_Carnival_2018_sum,holiday_Carnival_2018_max,holiday_Carnival_2018_mean,holiday_Carnival_2018_std,holiday_Carnival_2018_bool_purchase_percent,holiday_Good_Friday_2018_min,holiday_Good_Friday_2018_sum,holiday_Good_Friday_2018_max,holiday_Good_Friday_2018_mean,holiday_Good_Friday_2018_std,holiday_Good_Friday_2018_bool_purchase_percent,holiday_Easter_2018_min,holiday_Easter_2018_sum,holiday_Easter_2018_max,holiday_Easter_20

In [65]:
for x in [x for x in all_transactions if re.match('holiday_',x)]:
    groupped[x+'_min_sp'] = groupped[x+'_min'].values / groupped.purchase_amount_min.values
    groupped[x+'_sum_sp'] = groupped[x+'_sum'].values / groupped.purchase_amount_sum.values
    groupped[x+'_max_sp'] = groupped[x+'_max'].values / groupped.purchase_amount_max.values
    groupped[x+'_mean_sp'] = groupped[x+'_mean'].values / groupped.purchase_amount_mean.values
    
    groupped[x+'_min_hp'] = groupped[x+'_min'].values / groupped.all_holidays_min.values
    groupped[x+'_sum_hp'] = groupped[x+'_sum'].values / groupped.all_holidays_sum.values
    groupped[x+'_max_hp'] = groupped[x+'_max'].values / groupped.all_holidays_max.values
    groupped[x+'_mean_hp'] = groupped[x+'_mean'].values / groupped.all_holidays_mean.values

C:\Users\User\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [66]:
groupped['all_holidays_min_percent'] = groupped.all_holidays_min.values / groupped.purchase_amount_min.values
groupped['all_holidays_max_percent'] = groupped.all_holidays_max.values / groupped.purchase_amount_max.values
groupped['all_holidays_mean_percent'] = groupped.all_holidays_mean.values / groupped.purchase_amount_mean.values
groupped['all_holidays_sum_percent'] = groupped.all_holidays_sum.values / groupped.purchase_amount_sum.values
groupped['percent_in_holiday_days'] = groupped.day_with_holiday_purch_nunique.values / groupped.purchase_date_day_ptp_plus.values

In [67]:
groupped.head()

,card_id,holiday_New_Year_2017_min,holiday_New_Year_2017_sum,holiday_New_Year_2017_max,holiday_New_Year_2017_mean,holiday_New_Year_2017_std,holiday_New_Year_2017_bool_purchase_percent,holiday_Carnival_2017_min,holiday_Carnival_2017_sum,holiday_Carnival_2017_max,holiday_Carnival_2017_mean,holiday_Carnival_2017_std,holiday_Carnival_2017_bool_purchase_percent,holiday_Good_Friday_2017_min,holiday_Good_Friday_2017_sum,holiday_Good_Friday_2017_max,holiday_Good_Friday_2017_mean,holiday_Good_Friday_2017_std,holiday_Good_Friday_2017_bool_purchase_percent,holiday_Easter_2017_min,holiday_Easter_2017_sum,holiday_Easter_2017_max,holiday_Easter_2017_mean,holiday_Easter_2017_std,holiday_Easter_2017_bool_purchase_percent,holiday_Tirandes_Day_2017_min,holiday_Tirandes_Day_2017_sum,holiday_Tirandes_Day_2017_max,holiday_Tirandes_Day_2017_mean,holiday_Tirandes_Day_2017_std,holiday_Tirandes_Day_2017_bool_purchase_percent,holiday_Labor_Day_2017_min,holiday_Labor_Day_2017_sum,holiday_Labor_Day_2017_max,holiday_Labor_Day_2017_mean,holiday_Labor_Day_2017_std,holiday_Labor_Day_2017_bool_purchase_percent,holiday_Mothers_Day_2017_min,holiday_Mothers_Day_2017_sum,holiday_Mothers_Day_2017_max,holiday_Mothers_Day_2017_mean,holiday_Mothers_Day_2017_std,holiday_Mothers_Day_2017_bool_purchase_percent,holiday_Valentine_Day_2017_min,holiday_Valentine_Day_2017_sum,holiday_Valentine_Day_2017_max,holiday_Valentine_Day_2017_mean,holiday_Valentine_Day_2017_std,holiday_Valentine_Day_2017_bool_purchase_percent,holiday_Corpus_Christi_2017_min,holiday_Corpus_Christi_2017_sum,holiday_Corpus_Christi_2017_max,holiday_Corpus_Christi_2017_mean,holiday_Corpus_Christi_2017_std,holiday_Corpus_Christi_2017_bool_purchase_percent,holiday_Rebelion_day_2017_min,holiday_Rebelion_day_2017_sum,holiday_Rebelion_day_2017_max,holiday_Rebelion_day_2017_mean,holiday_Rebelion_day_2017_std,holiday_Rebelion_day_2017_bool_purchase_percent,holiday_Fathers_day_2017_min,holiday_Fathers_day_2017_sum,holiday_Fathers_day_2017_max,holiday_Fathers_day_2017_mean,holiday_Fathers_day_2017_std,holiday_Fathers_day_2017_bool_purchase_percent,holiday_Independence_day_2017_min,holiday_Independence_day_2017_sum,holiday_Independence_day_2017_max,holiday_Independence_day_2017_mean,holiday_Independence_day_2017_std,holiday_Independence_day_2017_bool_purchase_percent,holiday_Children_day_2017_min,holiday_Children_day_2017_sum,holiday_Children_day_2017_max,holiday_Children_day_2017_mean,holiday_Children_day_2017_std,holiday_Children_day_2017_bool_purchase_percent,holiday_All_Souls_day_2017_min,holiday_All_Souls_day_2017_sum,holiday_All_Souls_day_2017_max,holiday_All_Souls_day_2017_mean,holiday_All_Souls_day_2017_std,holiday_All_Souls_day_2017_bool_purchase_percent,holiday_Proclamation_of_Rep_2017_min,holiday_Proclamation_of_Rep_2017_sum,holiday_Proclamation_of_Rep_2017_max,holiday_Proclamation_of_Rep_2017_mean,holiday_Proclamation_of_Rep_2017_std,holiday_Proclamation_of_Rep_2017_bool_purchase_percent,holiday_Black_Friday_2017_min,holiday_Black_Friday_2017_sum,holiday_Black_Friday_2017_max,holiday_Black_Friday_2017_mean,holiday_Black_Friday_2017_std,holiday_Black_Friday_2017_bool_purchase_percent,holiday_Christmas_Day_2017_min,holiday_Christmas_Day_2017_sum,holiday_Christmas_Day_2017_max,holiday_Christmas_Day_2017_mean,holiday_Christmas_Day_2017_std,holiday_Christmas_Day_2017_bool_purchase_percent,holiday_New_Year_2018_min,holiday_New_Year_2018_sum,holiday_New_Year_2018_max,holiday_New_Year_2018_mean,holiday_New_Year_2018_std,holiday_New_Year_2018_bool_purchase_percent,holiday_Carnival_2018_min,holiday_Carnival_2018_sum,holiday_Carnival_2018_max,holiday_Carnival_2018_mean,holiday_Carnival_2018_std,holiday_Carnival_2018_bool_purchase_percent,holiday_Good_Friday_2018_min,holiday_Good_Friday_2018_sum,holiday_Good_Friday_2018_max,holiday_Good_Friday_2018_mean,holiday_Good_Friday_2018_std,holiday_Good_Friday_2018_bool_purchase_percent,holiday_Easter_2018_min,holiday_Easter_2018_sum,holiday_Easter_2018_max,holiday_Easter_20

In [68]:
feather.write_dataframe(groupped,'all_holidays_groupped')